### Como utilizar ML com Dados Não Estruturais Textuais?

* Técnicas de NLP preprocessam os dados para filtrarmos o que é mais relevante para a classificação 

![Title](../imgs/nlp-illustration.png)


* A tokenização nos auxilia a dividir strings em palavras
* Essas palavras serão as "features" que serão utilizadas para aprendizado de um modelo de Machine Learning
* Um algoritmo de Machine Learning só entende números (atributos sempre numéricos), sendo assim, após a tokenização, precisamos converter nossas palavras para valores numéricos
* O processo de transformação textual para base numérico denomina-se **bag of words**

![Title](../imgs/1-Bag-of-words.png)

* Técnicas como TF-IDF e CountVectorizer, apesar de eficientes, transformam o dataset em dado tabular
  * Cada coluna é uma palavra
  * Quando a palavra existir em uma dada amostra, substituimos o valor 0 pela frequência em que aquela palavra ocorre no texto
  
  <img src="https://www.researchgate.net/profile/Haider-Al-Khateeb/publication/291950178/figure/fig1/AS:330186932408324@1455734107458/Term-Frequency-Inverse-Document-Frequency-TF-IDF.png" width=800>
  
* A consequencia disso é a perda da relação semântica nas frases
* Perda de contexto das sentenças
* Vocabulário imenso (maldição da dimensionalidade)

### Word Embeddings to rescue!

* Vetor multidimensional de palavras
* Mapa de características com a frequência de palavras e considerações semânticas 
* Baseado em similaridade
* Aprendizado por meio de redes neurais

##### Como fazemos isso?
* Dada uma palavra da frase
* Pegar as palavras vizinhas 
* Rede Neural irá calcular a probabilidade dessa palavra vizinha ser a palavra a ser predita
* Selecionamos as palavras vizinhas definindo uma "janela" ou "kernel"
  * Janela = 2 considere duas palavras ao redor da palavra atual
![Title](../imgs/training_data.png)

* Criamos um corpus com as palavras organizadas de acordo com as janelas em torno do dataset.
* Palavras que aparecerem juntas tem maior probabilidade de terem significado semântico
  * Exemplo: "soviética" provavelmente terá uma semelhança com união
  
 ![Title](../imgs/vetor_multi.png)
 
[Leitura adicional](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/): funcionamento de uma rede neural para embeddings

### Arquiteturas Word Embeddings

* Existem diferentes formas de gerar nossas embeddings, gerando determinadas arquiteturas previamente conhecidos como CBOW e SkipGram. Essas arquiteturas também são denominadas como tipo word2vec (palavra para vetor)

* CBOW: predizer uma palavra baseado nas palavras vizinhas. Mais rápido, funciona melhor nas palavras mais frequentes.
* Skip-Gram: predizer o contexto baseado nas palavras vizinhas. Funciona melhor com datasets menores
<img src="https://leimao.github.io/images/article/2019-08-23-Word2Vec-Classic/word2vec.png">
* Ambas arquiteturas tem uma ambientação semelhante, modificando a forma como a saída e a entrada são organizadas
* Exemplo: Hoje vai fazer sol pela manhã com pancadas de chuva à tarde
* Utilizando CBOW\
![Title](../imgs/cbow.png)
* Utilizando Skip-Gram\
![Title](../imgs/skip_gram.png)

[Leitura adicional](https://www.tensorflow.org/tutorials/text/word2vec): transformando o corpus do CBOW/skip_gram em numérico

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Baixa as listas de stopwords e as tokenizações
nltk.download('stopwords')
nltk.download('punkt')

# Define as stopwords em inglês
sw_english = set(stopwords.words('english'))

# Instância o PorterStemmer
stemmer = PorterStemmer()

# Carrega o conjunto de dados
movies = pd.read_csv('../files/movies.csv', index_col = 0)

# Retira uma amostra do conjunto de dados
movies_sample = movies.sample(frac = 0.1, replace=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/Faaeel06/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/Faaeel06/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Cria um pipeline
def preprocessing(string):
    ###
    # Deixa apenas elementos alfanuméricos
    string = re.sub(r"[^a-zA-Z0-9]+", ' ', string)
    ###
    # deixa todas as palavras minúsculas
    string = string.lower()
    ###
    # tokenização
    words = word_tokenize(string)
    ###
    # Remove Stopwords
    filtered_words = []
    for w in words:
        if w not in sw_english:
            filtered_words.append(w)
    ###
    # Aplica o Stemming
    stem_words = []
    for w in filtered_words:
        s_words = stemmer.stem(w)
        stem_words.append(s_words)
    ###
    # Retorna a lista de palavras pré-processadas
    return stem_words

# Aplica o preprocessing nas críticas de filmes
movies_sample["filtered_words"] = movies_sample['text'].apply(lambda x: preprocessing(x))

In [4]:
movies_sample

,text,label,filtered_words
850,My only reason registering to this site was fo...,0,"[reason, regist, site, opportun, write, commen..."
8555,I really enjoyed this old black and white talk...,1,"[realli, enjoy, old, black, white, talki, firs..."
2150,"Time travel is theoretical, so I can give the ...",0,"[time, travel, theoret, give, script, leeway, ..."
35024,The Last Hunt is one of the few westerns ever ...,1,"[last, hunt, one, western, ever, made, deal, b..."
3795,It is disappointing to see as talented an acto...,0,"[disappoint, see, talent, actor, amitabh, bach..."
...,...,...,...
432,"Dr. Mordrid, what can I say? Jeffrey Combs has...",1,"[dr, mordrid, say, jeffrey, comb, done, br, br..."
15420,This show has shown it's true colors now that ...,0,"[show, shown, true, color, democrat, power, ne..."
317,This is an excellent but hard to find trippy W...,1,"[excel, hard, find, trippi, world, war, spi, t..."
23724,"I saw this film over Christmas, and what a gre...",1,"[saw, film, christma, great, film, tell, stori..."


### Treinamento de ML